### Chapter 05 트리 알고리즘 ▶️화이트 와인을 찾아라!
#### 05-3 트리의 앙상블 ▶️ 앙상블 학습을 알고 실습해 보기

------

#### - Random Forest : Ensemble 학슨의 대표주자 중 하나, 안정적 성능때문에 널리 사용됨 
#### - R/F는 여러개의 결정트리를 랜덤하게 만드는데, 이를 위해 훈련 데이터로 부터 랜덤하게 샘플을 추출한다. 
#### - 추출된 샘플을 Boostrap sample이라 하며, 보통 훈련세트의 크기와 같은 수의 Bootstrap sample을 추출한다. 
- ex> train dataset이 1000개라면, bootstrap 샘플은 랜덤하게 100개씩 10번 추출하는 식으로 1000개의 샘플을 만든다. 
- ex> 샘플 추출시 이때 처음 추출된 100개를 다시 원복시키고, 다음 샘플링을 진행함 (Booststrap의 원래 의미는 중복을 허용하는 데이터 샘플링임) 

#### - 사이킷런의 랜덤포레스트는 1) RandomForestClassifier와 2) RandomForestRegressor 
- ex> RandomForestClassifier는 가장 높은 확률을, RandomForestRegressor는 각 트리의 평균을 예측치로 사용함 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
wine = pd.read_csv("http://bit.ly/wine_csv_data")
print(wine.shape)
wine.head()

(6497, 4)


,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [11]:
## train, test dataset split 
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42)

#### 1. RandomForestClassifier를 실행하고 cross_validate()를 통해 교차검증 해보자 
- n_jobs = k : k개의 cpu core를 사용하라는 뜻, -1이면 가용한 모든 코어를 사용 
- return_train_score = True : 훈련세트에 대한 점수도 출력하라는 뜻 (검증세트에 대한 점수와 비교하기 위해) 

In [16]:
from sklearn.model_selection import cross_validate 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
score = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.9973541965122431
test_score = 0.8905151032797809


In [19]:
rf.fit(train_input, train_target)

print('features = alcohol, sugar, pH')
print('importances =' , rf.feature_importances_)

features = alcohol, sugar, pH
importances = [0.23167441 0.50039841 0.26792718]


#### - Decision Tree의 feature importances ([0.1234, 0.8686, 0.0079])대비, sugar의 중요도가 감소하고 alcole과 pH 중요도는 증가 
#### - 다수의 샘플을 훈련시킨 결과 -> RandomForest는 overfitting을 줄이고 모델일 일반화 시키는데 유용함 
------

#### 2. Extra Trees : Random Forest와 유사하나, Bootstrap을 사용하지 않고 전체 훈련세트를 무작위로 분할함 
- 사이킷런의 ExtraTreesClasslfier, ExtraTreesRegressor 

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.9973541965122431
test_score = 0.8905151032797809


In [22]:
et.fit(train_input, train_target)
print('features = alcohol, sugar, pH')
print('importances =' , et.feature_importances_)

features = alcohol, sugar, pH
importances = [0.20183568 0.52242907 0.27573525]


----

#### 3. Gradient Boosting 
- Overfitting 방지를 위해 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완 
- sklearn의 GradientBoostingClassifier : 깊이 3인 트리 100개를 사용 

In [38]:
from sklearn.ensemble import GradientBoostingClassifier 
gb = GradientBoostingClassifier(random_state = 42)

score = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.8881086892152563
test_score = 0.8720430147331015


In [39]:
gb.fit(train_input, train_target)
print('features = alcohol, sugar, pH')
print('importances =' , gb.feature_importances_)

features = alcohol, sugar, pH
importances = [0.11949946 0.74871836 0.13178218]


- overfitting은 되지 않았으나, 모형의 성능이 만족스럽지 못하다. 
- 훈련횟수를 좀더 늘여보자 ((tree의 갯수 100 -> 500)
- GradientBoost는 훈련횟수를 늘여도 overfitting에 강하다. 

In [40]:
gb2 = GradientBoostingClassifier(n_estimators = 500, random_state = 42)

score = cross_validate(gb2, train_input, train_target, return_train_score = True, n_jobs = -1)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.9215896435285418
test_score = 0.8749283704745687


In [41]:
gb2.fit(train_input, train_target)
print('features = alcohol, sugar, pH')
print('importances =' , gb2.feature_importances_)

features = alcohol, sugar, pH
importances = [0.14230296 0.70606994 0.1516271 ]


----
#### 4. Histogram-based Gradient Boosting
- GradientBoosting의 속도와 성능 개선을 위해 사용. 먼저 input값의 특성을 256개 구간으로 구분함 
- sklearn : HistGradientBoostingClassifier, HistGradientBoostingRegressor 제공하나...  
- XGBoost와 LightGMB을 주로 사용함 

In [45]:
#!pip install xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
score = cross_validate(xgb, train_input, train_target, return_train_score = True)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.9555033709953124
test_score = 0.8799326275264677


In [46]:
xgb.fit(train_input, train_target)
print('features = alcohol, sugar, pH')
print('importances =' , xgb.feature_importances_)

features = alcohol, sugar, pH
importances = [0.19727166 0.61559963 0.18712871]


In [47]:
#!pip install lightgbm
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state = 42)
score = cross_validate(lgbm, train_input, train_target, return_train_score = True, n_jobs = -1)

print('train_score =', np.mean(score['train_score']) )
print('test_score =', np.mean(score['test_score']) )

train_score = 0.935828414851749
test_score = 0.8801251203079884


In [48]:
lgbm.fit(train_input, train_target)
print('features = alcohol, sugar, pH')
print('importances =' , lgbm.feature_importances_)

features = alcohol, sugar, pH
importances = [ 909 1011 1080]


- [참고] LightGBM의 feature_importance 산정방식은 R/F나 XGBoost와 다르다. 
- https://hongl.tistory.com/344